### Library

In [2]:
from data.load_dataset import load_dataset
from data.merge_dataset import merge_dataset
from data.feature_engineering import *
from model.inference import save_csv
from model.feature_select import select_features
from model.data_split import split_features_and_target
from model.log_transformation import apply_log_transformation
from model.model_train import set_model, optuna_train
#from model.TreeModel import XGBoost
from pytorch_tabnet.tab_model import TabNetRegressor
import optuna
import torch

### Data load

In [3]:
# 기존 데이터 불러오기
train_data, test_data, sample_submission, interest_data, subway_data, school_data, park_data = load_dataset()
# 기존 데이터에 새로운 feature들을 병합한 데이터프레임 불러오기
train_data, test_data = merge_dataset(train_data, test_data, interest_data, subway_data, school_data, park_data)

### Data Preprocessing

In [4]:
# 위치 중복도 낮은 행 삭제
groups = train_data.groupby(["latitude", "longitude"])["index"].count()
conditioned_groups_index = groups[(groups >= 2) & (groups <= 5)].index # 이 범위를 파라미터로 조정하는걸로
small_groups = train_data[
    train_data["latitude"].isin(conditioned_groups_index.get_level_values(0)) &
    train_data["longitude"].isin(conditioned_groups_index.get_level_values(1))
]
train_data.drop(small_groups.index, axis=0, inplace=True)

# built_year > 2024 행 삭제
train_data = train_data[train_data["built_year"] < 2024]
train_data.reset_index(drop=True, inplace=True)

### Feature engineering

**Clustering**

In [5]:
# 데이터 준비
feature_columns = ["latitude", "longitude"]
coords = train_data[feature_columns]

# ClusteringModel 클래스 인스턴스 생성
clustering_model = ClusteringModel(data=coords)
kmeans_model = clustering_model.kmeans_clustering(n_clusters=25, 
                                                train_data=train_data, 
                                                test_data=test_data, 
                                                feature_columns=feature_columns, 
                                                label_column="region")

region_mean_prices = train_data.groupby("region")["deposit"].mean().reset_index()
region_mean_prices.columns = ["region", "mean_deposit"]
region_mean_prices["mean_deposit_category"] = region_mean_prices["mean_deposit"] // 10000

# train_data와 region_mean_prices 병합
train_data = train_data.merge(region_mean_prices, on="region", how="left")
test_data = test_data.merge(region_mean_prices, on="region", how="left")

**Log변환**

In [6]:
train_data, test_data = apply_log_transformation(train_data, test_data)

**Feature select**

In [7]:
train_data, test_data = select_features(train_data, test_data)

**train_data split**

In [8]:
X, y = split_features_and_target(train_data)

In [9]:
X.columns

Index(['log_area_m2', 'built_year', 'latitude', 'longitude',
       'log_subway_distance', 'log_school_distance', 'log_elementary_distance',
       'log_middle_distance', 'log_high_distance', 'log_park_distance',
       'park_area', 'park_area_above_300', 'contract_year_month',
       'contract_day', 'region', 'mean_deposit_category'],
      dtype='object')

In [10]:
y.columns

Index(['deposit', 'log_deposit'], dtype='object')

### Model Train and Evaluate

**Tabnet**
- 테이블 데이터에서도 딥러닝이 잘 동작할 수 있게 만들어진 모델
- 자동으로 중요한 features를 선택하기 떄문에 feature select부분은 제외

model_train.py 따라한 ver

In [14]:
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import KFold

# def cv_train(model, X: pd.DataFrame, y: pd.DataFrame, verbose: bool = True) -> float:
#     """K-Fold를 이용하여 Cross Validation을 수행하는 함수입니다.

#     Args:
#         model: 수행하려는 모델
#         X (pd.DataFrame): 독립 변수
#         y (pd.DataFrame): 예측 변수. deposit과 log_deposit 열로 나뉨.
#         verbose (bool, optional): Fold별 진행상황을 출력할지 여부. Defaults to True.

#     Returns:
#         float: 평균 MAE
#     """
#     cv = 5
#     kfold = KFold(n_splits=cv, shuffle=True, random_state=42)

#     mae_list = []
#     for i, (train_idx, valid_idx) in enumerate(kfold.split(X, y), start=1):
#         if verbose: print(f"training...[{i}/{cv}]")

#         X_train, y_train = X.loc[train_idx, :].values, y.loc[train_idx, "log_deposit"].values.reshape(-1, 1)
#         X_valid, y_valid = X.loc[valid_idx, :].values, y.loc[valid_idx, "deposit"].values.reshape(-1, 1)

#         model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=["mae"])

#         y_pred = model.predict(X_valid)
#         y_pred = np.expm1(y_pred)
#         fold_mae = mean_absolute_error(y_valid, y_pred)
#         if verbose: print(f"Valid MAE: {fold_mae:.4f}")
#         mae_list.append(fold_mae)

#     mae = np.mean(mae_list)
#     if verbose:
#         print("### K-fold Result ###")
#         print(f"Valid MAE: {mae:.4f}")
    
#     return mae

In [15]:
# def objective(trial):
#     params = {
#         "n_d": trial.suggest_int("n_d", 16, 64),
#         "n_steps": trial.suggest_int("n_steps", 3, 10),
#         "gamma": trial.suggest_float("gamma", 1.0, 2.0),
#         "lambda_sparse": trial.suggest_float("lambda_sparse", 0.0001, 0.01),
#         "optimizer_fn": torch.optim.Adam,
#         "optimizer_params": dict(lr=trial.suggest_float("learning_rate", 0.01, 0.1)),
#     }
#     model = TabNetRegressor(**params)
#     return cv_train(model, X, y, verbose=False)

합친 ver

In [16]:
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import KFold, cross_val_predict

# def objective(trial):
#     params = {
#         "n_d": trial.suggest_int("n_d", 8, 64),
#         "n_a": trial.suggest_int("n_a", 8, 64),
#         "n_steps": trial.suggest_int("n_steps", 3, 10),
#         "gamma": trial.suggest_float("gamma", 1.0, 2.0),
#         "lambda_sparse": trial.suggest_float("lambda_sparse", 0.0001, 0.01),
#         "optimizer_params": dict(lr=trial.suggest_float("learning_rate", 0.01, 0.1)),
#     }
    
#     # K-Fold 교차 검증
#     cv = 5
#     kfold = KFold(n_splits=cv, shuffle=True, random_state=42)
#     mae_list = []
    
#     for i, (train_idx, valid_idx) in enumerate(kfold.split(X, y), start=1):
#         if True: print(f"training...[{i}/{cv}]")
#         X_train, y_train = X.loc[train_idx, :].values, y.loc[train_idx, "log_deposit"].values.reshape(-1, 1)
#         X_valid, y_valid = X.loc[valid_idx, :].values, y.loc[valid_idx, "deposit"].values.reshape(-1, 1)
        
#         model = TabNetRegressor(**params)
#         # 모델 학습 (patience : 성능 개선되지않으면 early stopping)
#         model.fit(
#             X_train, y_train, 
#             eval_set=[(X_valid, y_valid)], 
#             eval_metric=["mae"], 
#             max_epochs=100,
#             patience=10,
#             batch_size=1024,
#             virtual_batch_size=128,
#         )
        
        
#         # 검증 데이터에 대한 예측
#         y_pred = model.predict(X_valid.values)
#         y_pred = np.expm1(y_pred)  # 로그 변환의 역변환
        
#         # MAE 계산
#         mae = mean_absolute_error(y_valid, y_pred) 
#         mae_list.append(mae)

#     # 교차 검증 후 MAE 평균값 반환
#     return np.mean(mae_list)

In [14]:
# import pandas as pd
# import numpy as np
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import KFold
# from model.TreeModel import XGBoost, LightGBM, CatBoost
# import optuna
# RANDOM_SEED = 42

# def set_model(model_name: str, **params):
#     """주어진 모델 이름에 따라 모델을 생성하고 반환하는 함수입니다.

#     Args:
#         model_name (str): 생성하려는 모델 이름
#         **params (dict): 모델 생성 시 사용할 하이퍼파라미터

#     Returns:
#         model (object): 생성된 모델 객체
#     """
#     match model_name:
#         case "xgboost":
#             model = XGBoost(**params)
#         case "lightgbm":
#             model = LightGBM(**params)
#         case "catboost":
#             model = CatBoost(**params)
#         case "tabnet":
#             model = TabNetRegressor(**params)
#     return model

# def cv_train(model, X: pd.DataFrame, y: pd.DataFrame, verbose: bool = True) -> float:
#     """K-Fold를 이용하여 Cross Validation을 수행하는 함수입니다.

#     Args:
#         model: 수행하려는 모델
#         X (pd.DataFrame): 독립 변수
#         y (pd.DataFrame): 예측 변수. deposit과 log_deposit 열로 나뉨.
#         verbose (bool, optional): Fold별 진행상황을 출력할지 여부. Defaults to True.

#     Returns:
#         float: 평균 MAE
#     """
#     cv = 5
#     kfold = KFold(n_splits=cv, shuffle=True, random_state=42)

#     mae_list = []
#     for i, (train_idx, valid_idx) in enumerate(kfold.split(X, y), start=1):
#         if verbose: print(f"training...[{i}/{cv}]")

#         X_train, y_train = X.loc[train_idx, :].values, y.loc[train_idx, "log_deposit"].values.reshape(-1,1)
#         X_valid, y_valid = X.loc[valid_idx, :].values, y.loc[valid_idx, "deposit"].values.reshape(-1,1)

#         model.fit(
#             X_train, y_train, 
#             eval_set=[(X_valid, y_valid)], 
#             eval_metric=["mae"], 
#             max_epochs=100,
#             patience=10
#         )

#         y_pred = model.predict(X_valid)
#         y_pred = np.expm1(y_pred)
#         fold_mae = mean_absolute_error(y_valid, y_pred)
#         if verbose: print(f"Valid MAE: {fold_mae:.4f}")
#         mae_list.append(fold_mae)

#     mae = np.mean(mae_list)
#     if verbose:
#         print("### K-fold Result ###")
#         print(f"Valid MAE: {mae:.4f}")
    
#     return mae

# def optuna_train(model_name: str, X: pd.DataFrame, y: pd.DataFrame) -> tuple[dict, float]:
#     """Optuna를 사용하여 주어진 모델의 하이퍼파라미터를 최적하는 함수

#     Args:
#         model_name (str): 최적화할 모델의 이름
#         X (pd.DataFrame): 독립 변수
#         y (pd.DataFrame): 예측 변수

#     Returns:
#         tuple[dict, float]:
#             - dict: 최적의 하이퍼파라미터
#             - float: 최적의 하이퍼파라미터에 대한 성능 지표(MAE)
#     """
#     def objective(trial):
#         match model_name:
#             case "xgboost":
#                 params = {
#                     "n_estimators": trial.suggest_int("n_estimators", 50, 300),
#                     "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
#                     "max_depth": trial.suggest_int("max_depth", 5, 12),
#                     "subsample": trial.suggest_float("subsample", 0.5, 1.0),
#                 }
#             case "lightgbm":
#                 params = {
#                     "verbose": -1,
#                     "n_estimators": trial.suggest_int("n_estimators", 50, 300),
#                     "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
#                     "max_depth": trial.suggest_int("max_depth", 5, 12),
#                     "subsample": trial.suggest_float("subsample", 0.5, 1.0),
#                     "num_leaves": trial.suggest_int("num_leaves", 20, 150),
#                     "objective": "regression_l1"
#             }
#             case "catboost":
#                 params = {
#                     "verbose": 0,
#                     "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
#                     "iterations": trial.suggest_int("iterations", 50, 500),
#                     "depth": trial.suggest_int("depth", 3, 10),
#                     "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 1, 10),
#                     # "bagging_temperature": trial.suggest_loguniform("bagging_temperature", 0.01, 1),
#                     # "border_count": trial.suggest_int("border_count", 32, 255),
#                     "cat_features": ["contract_day"],
#                     "task_type": "GPU",
#                     "devices": "cuda",
#                 }
#             case "tabnet":
#                 params = {
#                     "n_d": trial.suggest_int("n_d", 8, 64),
#                     "n_steps": trial.suggest_int("n_steps", 3, 10),
#                     "gamma": trial.suggest_float("gamma", 1.0, 2.0),
#                     "lambda_sparse": trial.suggest_float("lambda_sparse", 0.0001, 0.01),
#                     "optimizer_params": dict(lr=trial.suggest_float("learning_rate", 0.01, 0.1)),
#                 }
#         model = set_model(model_name, **params)
#         return cv_train(model, X, y, verbose=False)
    
#     sampler = optuna.samplers.TPESampler(seed=42)
#     study = optuna.create_study(direction="minimize", sampler=sampler)
#     study.optimize(objective, n_trials=50)
#     return study.best_params, study.best_value

값이 너무 커서 hyperparameter 범위 조정 & kfold train,valid split부분 수정

In [33]:
# import numpy as np
# import pandas as pd
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import KFold
# import optuna
# import torch
# from pytorch_tabnet.tab_model import TabNetRegressor

# def cv_train(model, X: pd.DataFrame, y: pd.DataFrame, verbose: bool = True) -> float:
#     """K-Fold를 이용하여 Cross Validation을 수행하는 함수입니다.

#     Args:
#         model: 수행하려는 모델
#         X (pd.DataFrame): 독립 변수
#         y (pd.DataFrame): 예측 변수. deposit과 log_deposit 열로 나뉨.
#         verbose (bool, optional): Fold별 진행상황을 출력할지 여부. Defaults to True.

#     Returns:
#         float: 평균 MAE
#     """
#     cv = 5
#     kfold = KFold(n_splits=cv, shuffle=True, random_state=42)

#     mae_list = []
#     for i, (train_idx, valid_idx) in enumerate(kfold.split(X, y), start=1):
#         if verbose: print(f"training...[{i}/{cv}]")

#         X_train, y_train = X.iloc[train_idx, :].values, y.iloc[train_idx, 0].values.reshape(-1, 1)
#         X_valid, y_valid = X.iloc[valid_idx, :].values, y.iloc[valid_idx, 1].values.reshape(-1, 1)


#         model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=["mae"])

#         y_pred = model.predict(X_valid)
#         y_pred = np.expm1(y_pred)  # 로그 변환 복구
        
#         fold_mae = mean_absolute_error(y_valid, y_pred)
#         if verbose: print(f"Valid MAE: {fold_mae:.4f}")
#         mae_list.append(fold_mae)

#     mae = np.mean(mae_list)
#     if verbose:
#         print("### K-fold Result ###")
#         print(f"Valid MAE: {mae:.4f}")
    
#     return mae

# def objective(trial):
#     params = {
#         "n_d": trial.suggest_int("n_d", 8, 64),
#         'n_a': trial.suggest_int('n_a', 8, 64),
#         "n_steps": trial.suggest_int("n_steps", 3, 10),
#         "gamma": trial.suggest_float("gamma", 1.0, 2.0),
#         'n_independent': trial.suggest_int('n_independent', 1, 5),
#         'n_shared': trial.suggest_int('n_shared', 1, 5),
#         "lambda_sparse": trial.suggest_float("lambda_sparse", 0.0001, 0.01),
#         "optimizer_fn": torch.optim.Adam,
#         "optimizer_params": dict(lr=trial.suggest_float("learning_rate", 0.001, 0.01)),
#     }
#     model = TabNetRegressor(**params)
#     return cv_train(model, X, y, verbose=False)


In [34]:
# sampler = optuna.samplers.TPESampler(seed=42)
# study = optuna.create_study(direction="minimize", sampler=sampler)
# study.optimize(objective, n_trials=50)

[I 2024-10-21 22:25:33,307] A new study created in memory with name: no-name-8a78beb2-0db3-402c-9c4e-566135980c95
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1136306929.78541| val_0_mae: 30751.69642|  0:01:39s
epoch 1  | loss: 206732459.15308| val_0_mae: 38290.00539|  0:03:21s
epoch 2  | loss: 162972446.67239| val_0_mae: 37552.91223|  0:05:02s


[W 2024-10-21 22:30:43,039] Trial 0 failed with parameters: {'n_d': 29, 'n_a': 62, 'n_steps': 8, 'gamma': 1.5986584841970366, 'n_independent': 1, 'n_shared': 1, 'lambda_sparse': 0.0006750277604651748, 'learning_rate': 0.008795585311974417} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_2977698/910628890.py", line 60, in objective
    return cv_train(model, X, y, verbose=False)
  File "/tmp/ipykernel_2977698/910628890.py", line 32, in cv_train
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=["mae"])
  File "/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py", line 258, in fit
    self._train_epoch(train_dataloader)
  File "/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py", line 489, in _train_epoch
    batch_logs = self.

KeyboardInterrupt: 

kfold 제거 ver 그러나 실패

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import optuna
import torch
from pytorch_tabnet.tab_model import TabNetRegressor

def train_model(model, X: pd.DataFrame, y: pd.DataFrame) -> float:
    """모델을 학습하고 검증 MAE를 계산하는 함수입니다.

    Args:
        model: 수행하려는 모델
        X (pd.DataFrame): 독립 변수
        y (pd.DataFrame): 예측 변수. deposit과 log_deposit 열로 나뉨.

    Returns:
        float: 검증 MAE
    """
    # 모델 학습
    model.fit(X.values, y.iloc[:, 0].values.reshape(-1, 1), eval_metric=["mae"])

    # 예측 및 로그 변환 복구
    y_pred = model.predict(X.values)
    y_pred = np.expm1(y_pred)  # log_deposit의 inverse log 처리

    # 검증 MAE 계산
    mae = mean_absolute_error(y.iloc[:, 1], y_pred)  # deposit 열을 사용
    return mae

def objective(trial):
    """Optuna를 이용하여 Hyperparameter 튜닝을 수행하는 함수입니다."""
    params = {
        "n_d": trial.suggest_int("n_d", 8, 64),
        "n_a": trial.suggest_int("n_a", 8, 64),
        "n_steps": trial.suggest_int("n_steps", 3, 10),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0),
        "n_independent": trial.suggest_int("n_independent", 1, 5),
        "n_shared": trial.suggest_int("n_shared", 1, 5),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 0.0001, 0.01),
        "optimizer_fn": torch.optim.Adam,
        "optimizer_params": dict(lr=trial.suggest_float("learning_rate", 0.001, 0.01)),
    }

    # TabNet 모델 생성
    model = TabNetRegressor(**params)
    
    # 모델 학습 및 MAE 계산
    mae = train_model(model, X, y)
    
    return mae

# Optuna 실험 세팅 및 실행
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(objective, n_trials=50)

# 최적 하이퍼파라미터 출력
print("Best hyperparameters: ", study.best_params)
print("Best MAE: ", study.best_value)


[I 2024-10-21 23:10:26,294] A new study created in memory with name: no-name-f1092e09-5d45-4f79-9193-04e1ad992f58
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 996308018.77346|  0:01:46s
epoch 1  | loss: 183449902.67735|  0:03:38s
epoch 2  | loss: 150613289.75286|  0:05:30s
epoch 3  | loss: 130629856.97483|  0:07:21s
epoch 4  | loss: 118432836.09611|  0:09:14s
epoch 5  | loss: 109284282.0|  0:11:06s
epoch 6  | loss: 104284150.86957|  0:12:58s
epoch 7  | loss: 99053404.99542|  0:14:50s
epoch 8  | loss: 95753091.6865|  0:16:42s
epoch 9  | loss: 93240958.37757|  0:18:34s
epoch 10 | loss: 91175232.80092|  0:20:32s
epoch 11 | loss: 89115790.45309|  0:22:29s
epoch 12 | loss: 87492333.18993|  0:24:20s
epoch 13 | loss: 86270799.08238|  0:26:13s
epoch 14 | loss: 85378919.91533|  0:28:05s
epoch 15 | loss: 84623095.04348|  0:29:56s
epoch 16 | loss: 83740571.10984|  0:31:51s
epoch 17 | loss: 83094334.98398|  0:33:45s
epoch 18 | loss: 82796808.69794|  0:35:38s
epoch 19 | loss: 82039606.3913|  0:37:29s
epoch 20 | loss: 81296512.66133|  0:39:30s
epoch 21 | loss: 81044328.27002|  0:41:31s
epoch 22 | loss: 80746521.58124|  0:43:33s
epoch 23 |

In [ ]:
# best_params = study.best_params
# print("Best parameters for Tabnet: ", best_params)

In [ ]:
best_model = TabNetRegressor(**best_params)
best_model.fit(X.values, y["log_deposit"].values.reshape(-1,1))

model_train.py에 합친다면 ver

In [ ]:
# best_params, mae = optuna_train("tabnet", X, y)
# best_model = set_model("tabnet", **best_params)
# best_model = best_model.fit(X.values, y["log_deposit"].values.reshape(-1,1))

### Inference

In [15]:
save_csv(best_model, test_data, sample_submission)